Instalação da Biblioteca Agentpy

In [ ]:
#Essa linha deve ser executada no Google Colab
pip install agentpy

Importações necessárias


In [ ]:
import agentpy as ap          # Biblioteca  para simulação baseada em agentes
import numpy as np            # Biblioteca para cálculos numéricos e vetoriais
import matplotlib.pyplot as plt  # Biblioteca para visualização e gráficos
import IPython                # Para exibir animações no Jupyter/Colab
import matplotlib as mpl      # Para ajustes gráficos adicionais

Tamanho da animação

In [ ]:
mpl.rcParams['animation.embed_limit'] = 2**28

Simulação -3 estimulos


> Simulação de Barbeiros em um ambiente natural ou controlado.
Onde o “Abrigo” é representado pelos dados de Odor de Barbeiro, “Camundongo” pelo Odor de Camundongo e “Calor” pelo Estímulo Térmico.



In [ ]:
# Função def para normalizar vetores
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

# Classe que define o agente "Barbeiro"
class Barbeiro(ap.Agent):
    def setup(self):
        self.velocity = normalize(self.model.nprandom.random(self.p.ndim) - 0.5) * self.p.velocidade_maxima
        self.estado = 'fuga' #Estado inicial de fuga
        self.ativo = True #Se o Barbeiro está ativo ou não
        self.tempo_parado = 0 # Contador de tempo que o barbeiro está parado
        self.saciado = False #Se o Barbeiro já se alimentou ou não
        self.tempo_na_comida = 0 # Tempo que passou no alimento

        # Tempo que o barbeiro vai permanecer em "fuga" antes de buscar estímulos
        if self.model.p.tipo_ambiente == 'controlado':
            min_fuga, max_fuga = self.p.tempo_fuga_controlado
        else:
            min_fuga, max_fuga = self.p.tempo_fuga_natural
        self.tempo_para_acalmar = self.model.nprandom.integers(min_fuga, max_fuga)

    # Define a posição inicial no espaço
    def setup_pos(self, space):
        self.space = space
        self.pos = space.positions[self]

    # Atualização da velocidade do barbeiro com base em: coesão, separação, alinhamento, bordas e estímulos
    def update_velocity(self):
        pos = self.pos; ndim = self.p.ndim
        v_coesao = np.zeros(ndim); v_separacao = np.zeros(ndim); v_alinhamento = np.zeros(ndim)
        #Regras de agrupamento
        vizinhos = self.space.neighbors(self, distance=self.p.raio_percepcao)
        if vizinhos:
            # Coesão: movimenta-se em direção ao centro do grupo
            centro_massa = np.mean(np.array(vizinhos.pos), axis=0)
            v_coesao = normalize(centro_massa - pos) * self.p.forca_coesao
            # Alinhamento: segue a velocidade média do grupo
            velocidade_media = np.mean(np.array(vizinhos.velocity), axis=0)
            v_alinhamento = normalize(velocidade_media) * self.p.forca_alinhamento
        # Separação: evita colisões com vizinhos muito próximos
        vizinhos_proximos = self.space.neighbors(self, distance=self.p.raio_separacao)
        if vizinhos_proximos:
            for vizinho in vizinhos_proximos: v_separacao += normalize(pos - vizinho.pos)
            v_separacao *= self.p.forca_separacao
        # Bordas da arena
        v_borda = np.zeros(ndim)
        forca_borda = self.p.forca_borda_fuga if self.estado == 'fuga' else self.p.forca_borda_normal
        dist_borda = self.p.distancia_borda
        for i in range(ndim):
            if pos[i] < dist_borda: v_borda[i] += forca_borda
            elif pos[i] > self.space.shape[i] - dist_borda: v_borda[i] -= forca_borda
        # Atração por estímulos (alimento, abrigo e calor)
        v_estimulos = np.zeros(ndim)
        if self.estado == 'busca':
            for nome, props in self.model.p.estimulos.items():
                # Ignora o estímulo de alimento se já estiver saciado
                if self.saciado and nome == 'Cheiro de Camundongo':
                    continue
                pos_estimulo = np.array(props['pos'])
                distancia = np.linalg.norm(pos - pos_estimulo)
                if distancia < props['raio_atracao']:
                    vetor_atracao = pos_estimulo - pos
                    forca = props['forca'] * (1 - distancia / props['raio_atracao'])
                    v_estimulos += normalize(vetor_atracao) * forca
        # Atualiza a velocidade total
        self.velocity += v_coesao + v_separacao + v_alinhamento + v_borda + v_estimulos
        self.velocity = normalize(self.velocity) * self.p.velocidade_maxima

    def update_position(self):
        # Move o barbeiro pela velocidade calculada
        self.space.move_by(self, self.velocity)

    def update_estado(self):
        # Muda de estado "fuga" → "busca" depois do tempo definido
        if self.estado == 'fuga' and self.model.t > self.tempo_para_acalmar:
            self.estado = 'busca'
       # Verifica se o barbeiro encontrou comida
        if not self.saciado:
            props_comida = self.model.p.estimulos['Cheiro de Camundongo']
            dist_comida = np.linalg.norm(self.pos - np.array(props_comida['pos']))

            if dist_comida < props_comida['raio_permanencia']:
                self.tempo_na_comida += 1

            if self.tempo_na_comida > 50: # Após 50 steps na comida, fica saciado
                self.saciado = True

    def update_parada(self):
        # Verifica se o barbeiro fica parado dentro do abrigo
        if not self.ativo: return
        props_agregacao = self.model.p.estimulos['Abrigo']
        dist = np.linalg.norm(self.pos - np.array(props_agregacao['pos']))
        if dist < props_agregacao['raio_permanencia'] and np.linalg.norm(self.velocity) < 0.1:
            self.tempo_parado += 1
        else:
            self.tempo_parado = 0
        # Após um tempo parado, o barbeiro não se move mais
        if self.tempo_parado > 20:
            self.ativo = False
            self.velocity = np.zeros(self.p.ndim)

# Classe que define o modelo da simulação
class SimuladorBarbeiros(ap.Model):
    def setup(self):
       # Cria o espaço bidimensional
        self.space = ap.Space(self, shape=[self.p.tamanho_arena] * self.p.ndim)
        # Cria a lista de agentes (barbeiros)
        self.agents = ap.AgentList(self, self.p.populacao, Barbeiro)
        # Adiciona os agentes ao espaço em posições aleatórias
        self.space.add_agents(self.agents, random=True)
        self.agents.setup_pos(self.space)

    def step(self):
        # Executa um passo de simulação: atualiza agentes ativos
        agentes_ativos = self.agents.select(self.agents.ativo == True)
        agentes_ativos.update_estado()
        agentes_ativos.update_velocity()
        agentes_ativos.update_position()
        agentes_ativos.update_parada()

    def update(self):
       # Condição de parada: nenhum agente ativo ou tempo máximo atingido
        if len(self.agents.select(self.agents.ativo == True)) == 0 or self.t >= self.p.steps:
            self.stop()

# Função que desenha um frame da animação
def animation_plot_single(m, ax):
    ax.clear()
    cores_estimulos = {'Abrigo': 'purple', 'Cheiro de Camundongo': 'green', 'Calor': 'orangered'}
    # Desenha estímulos como círculos no espaço
    for nome, props in m.p.estimulos.items():
        cor = cores_estimulos.get(nome, 'gray')
        circulo = plt.Circle(props['pos'], props['raio_permanencia'], color=cor, alpha=0.3)
        ax.add_patch(circulo)
        ax.text(props['pos'][0], props['pos'][1], nome.replace('Cheiro de ', ''),
                ha='center', va='center', fontsize=8, weight='bold', color='white')
    # Desenha os barbeiros
    for agent in m.agents:
        if not agent.ativo:
            ax.plot(agent.pos[0], agent.pos[1], marker='o', markersize=8, color='lightgray')
        else:
            cor_agente = 'gray' if agent.estado == 'fuga' else 'black'
            if agent.saciado:
                cor_agente = 'blue'
            ax.plot(agent.pos[0], agent.pos[1], marker='o', markersize=8, color=cor_agente, markeredgecolor='white', markeredgewidth=0.5)

    ax.set_title(f"Ambiente: {m.p.tipo_ambiente.upper()} - Tempo: {m.t}")
    ax.set_xlim(0, m.p.tamanho_arena); ax.set_ylim(0, m.p.tamanho_arena)
    ax.set_aspect('equal'); ax.set_axis_off()

# Função que cria a animação completa
def animation_plot(m, p):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=30))

parametros = {
    'tipo_ambiente': 'natural', # Tipo de ambiente: 'controlado' ou 'natural'
    'tamanho_arena': 100, 'ndim': 2, # Arena quadrada 100x100 em 2D
    'populacao': 20, # Número de barbeiros
    'steps': 1200, # Número máximo de passos de simulação
    'seed': 42, # Semente para reprodutibilidade
    'velocidade_maxima': 0.6, # Velocidade máxima dos agentes
    'raio_percepcao': 25, 'raio_separacao': 4, # Alcance de percepção e separação
    'forca_coesao': 0.04, 'forca_separacao': 0.3, 'forca_alinhamento': 0.2, # Regras de grupo
    'distancia_borda': 5, 'forca_borda_normal': 0.5, 'forca_borda_fuga': 1.0, # Influência da borda
    'tempo_fuga_natural': (50, 150), # Tempo de fuga em ambiente natural
    'tempo_fuga_controlado': (900, 950), #Tempo de fuga em ambiente controlado foi 933 segundos
    'estimulos': {
        'Abrigo': {'pos': [75, 50], 'raio_atracao': 16, 'raio_permanencia': 15, 'forca': 1.0},#raio de atração = quantidade de escolhas dos estimulos, duplicado para melhor visualização
        'Cheiro de Camundongo': {'pos': [25, 75], 'raio_atracao': 6, 'raio_permanencia': 12, 'forca': 0.36},# raio de permanencia = estímulo com um tempo de permanência maior deve ter seu raio de permanência para que o agente tenha mais espaço para interagir e permanecer.
        'Calor': {'pos': [25, 25], 'raio_atracao': 8, 'raio_permanencia': 12, 'forca': 0.26}#força = o cheiro do barbeiro foi o que teve maior média de segundos, sendo 253. Assim, os numeros colocados são de tempos médios divididos por 253.
    }
}

animation_plot(SimuladorBarbeiros, parametros)

#Simulação Simples - 2 estímulos

> Simulação de Barbeiros em um ambiente natural. Onde o “Abrigo” é representado pelos dados de Odor de Barbeiro e “Alimento”  é a representação dos estímulos Calor + Odor de Camundongo.




In [ ]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

class Barbeiro(ap.Agent):
    def setup(self):
        self.velocity = normalize(self.model.nprandom.random(self.p.ndim) - 0.5) * self.p.velocidade_maxima
        self.estado = 'fuga'
        self.ativo = True
        self.tempo_parado = 0
        self.saciado = False
        self.tempo_na_comida = 0

        if self.model.p.tipo_ambiente == 'controlado':
            min_fuga, max_fuga = self.p.tempo_fuga_controlado
        else:
            min_fuga, max_fuga = self.p.tempo_fuga_natural
        self.tempo_para_acalmar = self.model.nprandom.integers(min_fuga, max_fuga)


    def setup_pos(self, space):
        self.space = space
        self.pos = space.positions[self]

    def update_velocity(self):
        pos = self.pos; ndim = self.p.ndim
        v_coesao = np.zeros(ndim); v_separacao = np.zeros(ndim); v_alinhamento = np.zeros(ndim)
        vizinhos = self.space.neighbors(self, distance=self.p.raio_percepcao)
        if vizinhos:
            centro_massa = np.mean(np.array(vizinhos.pos), axis=0)
            v_coesao = normalize(centro_massa - pos) * self.p.forca_coesao
            velocidade_media = np.mean(np.array(vizinhos.velocity), axis=0)
            v_alinhamento = normalize(velocidade_media) * self.p.forca_alinhamento
        vizinhos_proximos = self.space.neighbors(self, distance=self.p.raio_separacao)
        if vizinhos_proximos:
            for vizinho in vizinhos_proximos: v_separacao += normalize(pos - vizinho.pos)
            v_separacao *= self.p.forca_separacao
        v_borda = np.zeros(ndim)
        forca_borda = self.p.forca_borda_fuga if self.estado == 'fuga' else self.p.forca_borda_normal
        dist_borda = self.p.distancia_borda
        for i in range(ndim):
            if pos[i] < dist_borda: v_borda[i] += forca_borda
            elif pos[i] > self.space.shape[i] - dist_borda: v_borda[i] -= forca_borda

        v_estimulos = np.zeros(ndim)
        if self.estado == 'busca':
            for nome, props in self.model.p.estimulos.items():
                if self.saciado and nome == 'Alimento':
                    continue
                pos_estimulo = np.array(props['pos'])
                distancia = np.linalg.norm(pos - pos_estimulo)
                if distancia < props['raio_atracao']:
                    vetor_atracao = pos_estimulo - pos
                    forca = props['forca'] * (1 - distancia / props['raio_atracao'])
                    v_estimulos += normalize(vetor_atracao) * forca

        self.velocity += v_coesao + v_separacao + v_alinhamento + v_borda + v_estimulos
        self.velocity = normalize(self.velocity) * self.p.velocidade_maxima

    def update_position(self):
        self.space.move_by(self, self.velocity)

    def update_estado(self):
        if self.estado == 'fuga' and self.model.t > self.tempo_para_acalmar:
            self.estado = 'busca'

        if not self.saciado:
            props_comida = self.model.p.estimulos['Alimento']
            dist_comida = np.linalg.norm(self.pos - np.array(props_comida['pos']))

            if dist_comida < props_comida['raio_permanencia']:
                self.tempo_na_comida += 1

            if self.tempo_na_comida > 50:
                self.saciado = True

    def update_parada(self):
        if not self.ativo: return
        props_agregacao = self.model.p.estimulos['Abrigo']
        dist = np.linalg.norm(self.pos - np.array(props_agregacao['pos']))
        if dist < props_agregacao['raio_permanencia'] and np.linalg.norm(self.velocity) < 0.1:
            self.tempo_parado += 1
        else:
            self.tempo_parado = 0
        if self.tempo_parado > 20:
            self.ativo = False
            self.velocity = np.zeros(self.p.ndim)

class SimuladorBarbeiros(ap.Model):
    def setup(self):
        self.space = ap.Space(self, shape=[self.p.tamanho_arena] * self.p.ndim)
        self.agents = ap.AgentList(self, self.p.populacao, Barbeiro)
        self.space.add_agents(self.agents, random=True)
        self.agents.setup_pos(self.space)

    def step(self):
        agentes_ativos = self.agents.select(self.agents.ativo == True)
        agentes_ativos.update_estado()
        agentes_ativos.update_velocity()
        agentes_ativos.update_position()
        agentes_ativos.update_parada()

    def update(self):
        if len(self.agents.select(self.agents.ativo == True)) == 0 or self.t >= self.p.steps:
            self.stop()

def animation_plot_single(m, ax):
    ax.clear()
    cores_estimulos = {'Abrigo': 'purple', 'Alimento': 'green'}
    for nome, props in m.p.estimulos.items():
        cor = cores_estimulos.get(nome, 'gray')
        circulo = plt.Circle(props['pos'], props['raio_permanencia'], color=cor, alpha=0.3)
        ax.add_patch(circulo)
        ax.text(props['pos'][0], props['pos'][1], nome.replace('Cheiro de ', ''),
                ha='center', va='center', fontsize=8, weight='bold', color='white')

    for agent in m.agents:
        if not agent.ativo:
            ax.plot(agent.pos[0], agent.pos[1], marker='o', markersize=8, color='lightgray')
        else:
            cor_agente = 'gray' if agent.estado == 'fuga' else 'black'
            if agent.saciado:
                cor_agente = 'blue'
            ax.plot(agent.pos[0], agent.pos[1], marker='o', markersize=8, color=cor_agente, markeredgecolor='white', markeredgewidth=0.5)

    ax.set_title(f"Ambiente: {m.p.tipo_ambiente.upper()} - Tempo: {m.t}")
    ax.set_xlim(0, m.p.tamanho_arena); ax.set_ylim(0, m.p.tamanho_arena)
    ax.set_aspect('equal'); ax.set_axis_off()

def animation_plot(m, p):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=30))

parametros = {
    'tipo_ambiente': 'natural',
    'tamanho_arena': 100, 'ndim': 2, 'populacao': 20,
    'steps': 1200,
    'seed': 42,
    'velocidade_maxima': 0.6,
    'raio_percepcao': 25, 'raio_separacao': 4,
    'forca_coesao': 0.04, 'forca_separacao': 0.3, 'forca_alinhamento': 0.2,
    'distancia_borda': 5, 'forca_borda_normal': 0.5, 'forca_borda_fuga': 1.0,
    'tempo_fuga_natural': (50, 150),
    'tempo_fuga_controlado': (900, 950),
    'estimulos': {
        'Abrigo': {'pos': [75, 50], 'raio_atracao': 16, 'raio_permanencia': 15, 'forca': 1.0},
        'Alimento': {'pos': [25, 75], 'raio_atracao': 14, 'raio_permanencia': 14, 'forca': 0.62}}
}

animation_plot(SimuladorBarbeiros, parametros)